In [2]:
import _init_paths

import tensorflow as tf
import numpy as np
import nengo_dl
import nengo 
import matplotlib.pyplot as plt

from configs.exp_configs import tf_exp_cfg as tf_cfg, nengo_dl_cfg as ndl_cfg
from utils.cnn_2d_utils import get_2d_cnn_model, get_2d_cnn_model_without_dropouts
#from utils.nengo_dl_utils import get_nengo_dl_model
from utils.base_utils.exp_utils import get_grouped_slices_2d_pooling, get_grouped_slices_2d_pooling_cl
from utils.base_utils.data_prep_utils import get_batches_of_exp_dataset, get_exp_dataset
from utils.results_prep_utils.analyse_results import (get_majority_voted_class, get_accuracy_via_majority_voting,
                                                      get_max_pool_otpt_indices)

In [4]:
model = get_2d_cnn_model((32, 32, 3), tf_cfg) # Channels first shape (1, 28, 28).
model[0].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 4)         12        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 32)        1152      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18432 

In [7]:
new_model = get_2d_cnn_model_without_dropouts(model[0])
new_model[0].summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 4)         12        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 32)        1152      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18432     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 64)        3686

In [ ]:
ndl_model = nengo_dl.Converter(model[0])

In [ ]:
print(dir(ndl_model))

In [8]:
for lyr in ndl_model.model.layers:
  if lyr.name in ["conv2d", "conv2d_1"]:
    print(lyr.output.shape[1:])

(16, 26, 26)
(24, 11, 11)


In [20]:
nengo_dl.__version__

'3.4.0'

In [2]:
grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=1, rows=5, cols=5)

In [3]:
len(grouped_slices) # 10816 == 16 x 26 x 26

25

In [6]:
grouped_slices[:16]

array([ 0,  1,  5,  6,  2,  3,  7,  8, 10, 11, 15, 16, 12, 13, 17, 18])

In [5]:
grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=24, rows=11, cols=11)

In [6]:
len(grouped_slices)# 24 x 11 x 11 = 2904 

# ###############################################################

In [2]:
ndl_model, ngo_probes_lst = get_nengo_dl_model(
    (1, 28, 28), tf_cfg, ndl_cfg, mode="test", num_clss=10, max_to_avg_pool=False)

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2ae894efbb90> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


In [8]:
ndl_model.net.all_connections[3].post_obj.size_in

21632

In [4]:
for i, conn in enumerate(ndl_model.net.all_connections):
  if (isinstance(conn.post_obj, nengo_dl.tensor_node.TensorNode) and
      conn.post_obj.label.startswith("max_pooling")):
    print(conn, ndl_model.net.all_connections[i])

<Connection from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">> <Connection from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">>
<Connection from <Neurons of <Ensemble "conv2d_1.0">> to <TensorNode "max_pooling2d_1">> <Connection from <Neurons of <Ensemble "conv2d_1.0">> to <TensorNode "max_pooling2d_1">>


In [5]:
ndl_model.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 26, 26)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 13, 13)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 11, 11)        3480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 5, 5)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 3, 3)          6944      
_________________________________________________________________
flatten (Flatten)            (None, 288)               0     

In [4]:
def get_grouped_slices_2d_pooling(**kwargs):
  """
  Creates square grouped slices based on the `pool_size`. E.g. for a flattened
  array, the indices are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
  ...] with `pool_size` = (2, 2), `rows=4`, `cols=4`, the flattened array is
  actually:
  [[0, 1, 2, 3],
   [4, 5, 6, 7],
   [8, 9, 10, 11],
   [12, 13, 14, 15]]

  so the returned grouped slices array should be of indices:
  [0, 1, 4, 5, 2, 3, 6, 7, 8, 9, 12, 13, 10, 11, 14, 15, ...]

  Note: It expects the "Channels First" coding of Input/Conv layers. It is also
  assumed that the input `rows` and `cols` are of same size and `pool_size` also
  has same row/col dimension.

  Args:
    kwargs <dict>:
      pool_size <tuple>: (int, int) for 2D Pooling - (row, col) arrangement.
      num_chnls <int>: Number of channels in the reshaped matrix.
      rows <int>: Number of rows in the reshaped matrix.
      cols <int>: Number of columns in the reshaped matrix.

  Returns:
    <[int]>
  """
  pool_size, num_chnls, rows, cols = (
      kwargs["pool_size"], kwargs["num_chnls"], kwargs["rows"], kwargs["cols"])
  matrix = np.arange(rows * cols * num_chnls).reshape((num_chnls, rows, cols))
  grouped_slices = np.zeros(rows * cols * num_chnls, dtype=int)
  start, slice_len = 0, np.prod(pool_size)

  for chnl in range(num_chnls):
    for row in range(rows//pool_size[0]):
      for col in range(cols//pool_size[1]):
        grouped_slices[start:start+slice_len] = (
            matrix[chnl, row*pool_size[0]:row*pool_size[0]+pool_size[0],
                   col*pool_size[1]:col*pool_size[1]+pool_size[1]]).flatten()
        start += slice_len
        
  return grouped_slices

In [5]:
get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=4, rows=3, cols=3)

array([ 0,  1,  3,  4,  9, 10, 12, 13, 18, 19, 21, 22, 27, 28, 30, 31,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [2]:
def get_grouped_slices_2d_pooling(is_channels_last=False, **kwargs):
  """
  Creates square grouped slices based on the `pool_size`. E.g. for a flattened
  array, the indices are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
  ...] with `pool_size` = (2, 2), `rows=4`, `cols=4`, the flattened array is
  actually:
  [[0, 1, 2, 3],
   [4, 5, 6, 7],
   [8, 9, 10, 11],
   [12, 13, 14, 15]]

  so the returned grouped slices array should be of indices:
  [0, 1, 4, 5, 2, 3, 6, 7, 8, 9, 12, 13, 10, 11, 14, 15, ...]

  Note: It expects the "Channels First" coding of Input/Conv layers. It is also
  assumed that the input `rows` and `cols` are of same size and `pool_size` also
  has same row/col dimension.

  Args:
    kwargs <dict>:
      pool_size <tuple>: (int, int) for 2D Pooling - (row, col) arrangement.
      num_chnls <int>: Number of channels in the reshaped matrix.
      rows <int>: Number of rows in the reshaped matrix.
      cols <int>: Number of columns in the reshaped matrix.

  Returns:
    <[int]>
  """
  pool_size, num_chnls, rows, cols = (
      kwargs["pool_size"], kwargs["num_chnls"], kwargs["rows"], kwargs["cols"])
  matrix = np.arange(rows * cols * num_chnls).reshape((rows, cols, num_chnls))
#   if is_channels_last:
#     matrix = np.moveaxis(matrix, 0, -1)
  grouped_slices = np.zeros(rows * cols * num_chnls, dtype=int)
  start, slice_len = 0, np.prod(pool_size)

  for chnl in range(num_chnls):
    for row in range(rows//pool_size[0]):
      for col in range(cols//pool_size[1]):
#         if is_channels_last:
#           grouped_slices[start:start+slice_len] = (
#               matrix[row*pool_size[0]:row*pool_size[0]+pool_size[0],
#               col*pool_size[1]:col*pool_size[1]+pool_size[1], chnl]).flatten()
#         else:
          grouped_slices[start:start+slice_len] = (
              matrix[row*pool_size[0]:row*pool_size[0]+pool_size[0],
              col*pool_size[1]:col*pool_size[1]+pool_size[1], chnl]).flatten()
          start += slice_len
        
  return grouped_slices

In [4]:
matrix_cl = np.arange(3 * 3 * 4).reshape((4, 3, 3))
print(matrix_cl)
print("*"*80)
matrix_cl = np.moveaxis(matrix_cl, 0, -1)
print(matrix_cl[:, :, 0])
print("*"*80)
flattened = matrix_cl.flatten()
print(flattened)
print("*"*80)
cl_grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=4, rows=3, cols=3)
print(flattened[cl_grouped_slices])
print("*"*80)
print(flattened.reshape(3, 3, 4)[:, :, 0])

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]

 [[27 28 29]
  [30 31 32]
  [33 34 35]]]
********************************************************************************
[[0 1 2]
 [3 4 5]
 [6 7 8]]
********************************************************************************
[ 0  9 18 27  1 10 19 28  2 11 20 29  3 12 21 30  4 13 22 31  5 14 23 32
  6 15 24 33  7 16 25 34  8 17 26 35]
********************************************************************************
[ 0  1  3  4  9 10 12 13 18 19 21 22 27 28 30 31  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
********************************************************************************
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [5]:
matrix = np.arange(3* 3 * 4).reshape((3, 3, 4))
print(matrix[:, :, 0])

[[ 0  4  8]
 [12 16 20]
 [24 28 32]]


In [28]:
cl_grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=4, rows=3, cols=3)
cl_grouped_slices

array([ 0,  4, 12, 16,  1,  5, 13, 17,  2,  6, 14, 18,  3,  7, 15, 19,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [7]:
get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=4, rows=3, cols=3, is_channels_last=True)

array([ 0,  1,  3,  4,  9, 10, 12, 13, 18, 19, 21, 22, 27, 28, 30, 31,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [16]:
matrix_cl = np.moveaxis(matrix, 0, -1)
print(matrix_cl.shape)
print(matrix_cl)

(3, 3, 4)
[[[ 0  9 18 27]
  [ 1 10 19 28]
  [ 2 11 20 29]]

 [[ 3 12 21 30]
  [ 4 13 22 31]
  [ 5 14 23 32]]

 [[ 6 15 24 33]
  [ 7 16 25 34]
  [ 8 17 26 35]]]


In [19]:
flattened = matrix_cl.flatten()
flattened

array([ 0,  9, 18, 27,  1, 10, 19, 28,  2, 11, 20, 29,  3, 12, 21, 30,  4,
       13, 22, 31,  5, 14, 23, 32,  6, 15, 24, 33,  7, 16, 25, 34,  8, 17,
       26, 35])

In [29]:
flattened[cl_grouped_slices]

array([ 0,  1,  3,  4,  9, 10, 12, 13, 18, 19, 21, 22, 27, 28, 30, 31,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [30]:
reshaped_matrix_cl = np.reshape(flattened, (3, 3, 4))
reshaped_matrix_cl

array([[[ 0,  9, 18, 27],
        [ 1, 10, 19, 28],
        [ 2, 11, 20, 29]],

       [[ 3, 12, 21, 30],
        [ 4, 13, 22, 31],
        [ 5, 14, 23, 32]],

       [[ 6, 15, 24, 33],
        [ 7, 16, 25, 34],
        [ 8, 17, 26, 35]]])

In [48]:
grouped_slices_cl = get_grouped_slices_2d_pooling_cl(pool_size=(2, 2), rows=4, cols=4, num_chnls=2)
grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), rows=4, cols=4, num_chnls=2)

In [49]:
matrix = np.arange(4 * 4 * 2).reshape((2, 4, 4))
print(matrix)
print("*"*80)
matrix_cl = np.moveaxis(matrix, 0, -1)
print(matrix_cl.shape)
print(matrix_cl[:, :, 0])
print("*"*80)
flattened = matrix_cl.flatten()

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]]

 [[16 17 18 19]
  [20 21 22 23]
  [24 25 26 27]
  [28 29 30 31]]]
********************************************************************************
(4, 4, 2)
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
********************************************************************************


In [50]:
flattened[grouped_slices_cl]

array([ 0,  1,  4,  5,  2,  3,  6,  7,  8,  9, 12, 13, 10, 11, 14, 15, 16,
       17, 20, 21, 18, 19, 22, 23, 24, 25, 28, 29, 26, 27, 30, 31])

In [51]:
matrix.shape

(2, 4, 4)

In [55]:
flattened = matrix.flatten()

In [57]:
flattened[grouped_slices]

array([ 0,  1,  4,  5,  2,  3,  6,  7,  8,  9, 12, 13, 10, 11, 14, 15, 16,
       17, 20, 21, 18, 19, 22, 23, 24, 25, 28, 29, 26, 27, 30, 31])

# Checking majority voting accuracy

In [3]:
file_path = ("/home/rgaurav/projects/def-anarayan/rgaurav/SpikingInvestigation/"
             "misc/Checking_Coversion_of_Dropout_and_BN_layers/")
res = np.load(file_path+"NDL_pred_output_epochs_128_sfr_400.npy")

In [4]:
get_accuracy_via_majority_voting(res, 1, "cifar10")

81.05

# Checking the NengoDL MaxPool outputs (first 1000 images)

In [10]:
mp1_otpt = np.load(file_path + "NDL_pred_output_epochs_96_sfr_400_mp1_actvns.npy")
mp2_otpt = np.load(file_path + "NDL_pred_output_epochs_96_sfr_400_mp2_actvns.npy")

In [11]:
mp1_otpt.shape, mp2_otpt.shape

((1000, 120, 6272), (1000, 120, 1600))

In [12]:
np.random.seed(0)
test_img_idcs = np.random.randint(0, 1000, 10)
print(test_img_idcs)
mp_ti_otpt_idcs = get_max_pool_otpt_indices(mp1_otpt)

[684 559 629 192 835 763 707 359   9 723]


In [13]:
np.random.seed(0)
test_img_idcs = np.random.randint(0, 1000, 10)
print(test_img_idcs)

[684 559 629 192 835 763 707 359   9 723]


In [21]:
# for ti in test_img_idcs[:1]:
#     for mp_otpt_idx in mp_ti_otpt_idcs[ti]:
#         print(mp1_otpt[ti, :, mp_otpt_idx])

[0.        0.        0.        0.        0.        0.        0.
 0.        2.4999998 0.        0.        0.        2.4999998 0.
 0.        2.4999998 2.4999998 0.        0.        2.4999998 2.4999998
 0.        0.        2.4999998 2.4999998 0.        0.        2.4999998
 2.4999998 0.        2.4999998 2.4999998 2.4999998 2.4999998 0.
 0.        2.4999998 2.4999998 2.4999998 0.        2.4999998 2.4999998
 0.        2.4999998 2.4999998 2.4999998 2.4999998 0.        0.
 2.4999998 2.4999998 0.        0.        2.4999998 2.4999998 2.4999998
 0.        2.4999998 2.4999998 0.        2.4999998 2.4999998 2.4999998
 0.        2.4999998 2.4999998 2.4999998 0.        2.4999998 0.
 2.4999998 2.4999998 0.        2.4999998 2.4999998 2.4999998 0.
 2.4999998 2.4999998 0.        2.4999998 2.4999998 0.        0.
 2.4999998 2.4999998 0.        0.        2.4999998 2.4999998 0.
 2.4999998 2.4999998 2.4999998 2.4999998 2.4999998 2.4999998 2.4999998
 2.4999998 2.4999998 2.4999998 0.        2.4999998 0.        2

 2.4999998]
[0.        0.        0.        0.        0.        0.        0.
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        2.4999998 0.        0.
 0.        0.        0.        0.        0.        0.        0.
 2.4999998 0.        2.4999998 0.        0.        0.        0.
 2.4999998 0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        2.4999998 0.        0.        0.
 0.        2.4999998 0.        0.        0.        0.        0.
 2.4999998 0.        0.        0.        0.        2.4999998 0.
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.

[0.        0.        0.        0.        0.        0.        0.
 0.        0.        0.        0.        0.        0.        0.
 2.4999998 0.        2.4999998 0.        0.        0.        0.
 0.        0.        0.        2.4999998 0.        0.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        0.        0.        0.        2.4999998 0.
 0.        0.        2.4999998 2.4999998 0.        0.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        2.4999998 0.        0.        0.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.        2.4999998 0.        0.        0.        0.        0.
 2.4999998 0.        0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.        0.        2.4999998
 0.        0.      

[0.        0.        0.        0.        0.        0.        0.
 0.        0.        0.        0.        2.4999998 2.4999998 0.
 0.        2.4999998 0.        0.        0.        0.        2.4999998
 2.4999998 0.        0.        2.4999998 2.4999998 0.        0.
 0.        2.4999998 2.4999998 0.        0.        2.4999998 2.4999998
 0.        2.4999998 0.        2.4999998 0.        2.4999998 2.4999998
 0.        0.        0.        2.4999998 2.4999998 0.        0.
 0.        2.4999998 2.4999998 0.        0.        2.4999998 0.
 2.4999998 0.        2.4999998 0.        2.4999998 0.        0.
 2.4999998 2.4999998 0.        0.        2.4999998 0.        0.
 2.4999998 2.4999998 0.        0.        2.4999998 0.        0.
 2.4999998 2.4999998 0.        2.4999998 0.        0.        0.
 0.        2.4999998 0.        0.        0.        0.        2.4999998
 0.        0.        2.4999998 2.4999998 0.        0.        0.
 0.        2.4999998 0.        0.        2.4999998 2.4999998 0.
 0.        0

[0.        0.        0.        0.        0.        0.        0.
 0.        2.4999998 2.4999998 0.        2.4999998 0.        0.
 2.4999998 0.        2.4999998 2.4999998 2.4999998 2.4999998 2.4999998
 0.        2.4999998 0.        0.        2.4999998 0.        0.
 2.4999998 0.        0.        2.4999998 2.4999998 0.        2.4999998
 0.        2.4999998 2.4999998 0.        0.        0.        2.4999998
 2.4999998 2.4999998 0.        0.        0.        2.4999998 2.4999998
 2.4999998 2.4999998 2.4999998 2.4999998 0.        2.4999998 0.
 2.4999998 0.        2.4999998 2.4999998 0.        0.        0.
 2.4999998 2.4999998 2.4999998 0.        0.        0.        2.4999998
 2.4999998 0.        0.        2.4999998 0.        0.        2.4999998
 0.        0.        2.4999998 2.4999998 2.4999998 0.        2.4999998
 2.4999998 2.4999998 2.4999998 2.4999998 0.        0.        0.
 2.4999998 2.4999998 0.        2.4999998 0.        0.        2.4999998
 2.4999998 0.        2.4999998 0.        2.49999

[0.        0.        0.        0.        0.        0.        0.
 0.        0.        0.        2.4999998 2.4999998 0.        2.4999998
 2.4999998 0.        2.4999998 0.        0.        0.        0.
 2.4999998 0.        0.        0.        2.4999998 0.        0.
 2.4999998 0.        0.        0.        0.        0.        0.
 0.        2.4999998 0.        0.        0.        0.        0.
 0.        2.4999998 0.        0.        0.        0.        2.4999998
 0.        0.        0.        0.        0.        0.        0.
 0.        2.4999998 0.        0.        0.        0.        0.
 0.        0.        0.        2.4999998 2.4999998 0.        0.
 0.        0.        0.        0.        0.        0.        0.
 2.4999998 0.        0.        0.        2.4999998 0.        0.
 2.4999998 0.        0.        0.        0.        0.        0.
 2.4999998 0.        0.        0.        0.        0.        0.
 0.        0.        2.4999998 0.        0.        0.        0.
 0.        0.        0.   

In [40]:
n

(array([0]),)